In [1]:
!pip install datasets


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install accelerate


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install transformers[torch]


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [19]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
import torch

# 1. Load the text data from a .txt file
def load_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text_data = file.read()
    return text_data

# Replace 'sample.txt' with your .txt file path
text_data = load_text_from_file('data.txt')

# 2. Split the data into chunks for training (optional, depending on text length)
# GPT-2 has a max token length of 1024, so split the data into chunks of appropriate size
def split_text(text, chunk_size=1024):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

chunks = split_text(text_data)

# 3. Tokenization
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Tokenize the text and add labels (which are shifted input_ids)
    tokens = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=1024)
    tokens["labels"] = tokens["input_ids"].copy()  # Labels should be the same as input_ids
    return tokens
data = {"text": chunks}
dataset = Dataset.from_dict(data)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# 5. Define TrainingArguments for fine-tuning
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=200,
    per_device_train_batch_size=4,  # Adjust according to GPU memory
    per_device_eval_batch_size=4,  # Adjust according to GPU memory
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,  # Only compute the loss
)

# 6. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 7. Train the model
trainer.train()

# 8. Save the fine-tuned model
model.save_pretrained('./fine-tuned-gpt2')
tokenizer.save_pretrained('./fine-tuned-gpt2')


Map:   0%|          | 0/222 [00:00<?, ? examples/s]

Step,Training Loss
100,0.817000
200,0.579700
300,0.513000
400,0.464300
500,0.416700
600,0.370200
700,0.330200
800,0.290100
900,0.256800
1000,0.223200


('./fine-tuned-gpt2/tokenizer_config.json',
 './fine-tuned-gpt2/special_tokens_map.json',
 './fine-tuned-gpt2/vocab.json',
 './fine-tuned-gpt2/merges.txt',
 './fine-tuned-gpt2/added_tokens.json')

In [7]:
!pip install huggingface_hub


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [20]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('./fine-tuned-gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('./fine-tuned-gpt2')

model.save_pretrained('./fine-tuned-gpt2')
tokenizer.save_pretrained('./fine-tuned-gpt2')

model.push_to_hub("zulqarnain-kernel/mokalma-gpt2")
tokenizer.push_to_hub("zulqarnain-kernel/mokalma-gpt2")


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/zulqarnain-kernel/mokalma-gpt2/commit/05003e0b7de3e9c4236c80c23743e7f4804a414d', commit_message='Upload tokenizer', commit_description='', oid='05003e0b7de3e9c4236c80c23743e7f4804a414d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/zulqarnain-kernel/mokalma-gpt2', endpoint='https://huggingface.co', repo_type='model', repo_id='zulqarnain-kernel/mokalma-gpt2'), pr_revision=None, pr_num=None)

In [22]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("zulqarnain-kernel/mokalma-gpt2")  # Load from Hugging Face Hub
tokenizer = GPT2Tokenizer.from_pretrained("zulqarnain-kernel/mokalma-gpt2")  # Load from Hugging Face Hub

# Alternatively, load from the local path if it's saved locally:
# model = GPT2LMHeadModel.from_pretrained('./fine-tuned-gpt2')
# tokenizer = GPT2Tokenizer.from_pretrained('./fine-tuned-gpt2')

# Text generation function
def generate_text(prompt, max_length=1028, num_return_sequences=1):
    # Encode the prompt text
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Generate text
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=1.0
    )

    # Decode and print the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Example usage
prompt = "You know they're sellin' masks."
generated_text = generate_text(prompt, max_length=1028)
print("Generated Text:\n", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text:
 You know they're sellin' masks.
  They crack up when I see them. Start making fun of
him. I tell him to shut up.

 37  EXT. ROADWAY, POTTER'S FIELD CEMETERY - DUSK 
37 -  INT. CITY HOSPITAL - MORNING 38

 Joker is on a overcrowded street. Bloody 

 -40 INT. WGCTV STUDIOS, STREET - CONTINUOUS 
